In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from os import listdir
from os.path import isfile, join
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/simone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import wordCountSentimentFull as wordCountSentiment

In [3]:
dictionary = pd.read_csv('LoughranMcDonald_MyDictionary2.csv')
dictionary.drop('Unnamed: 0', 1, inplace=True)

dictionary.head()

,NEGATIVE,POSITIVE,UNCERTAINTY,LITIGIOUS,CONSTRAINING,SUPERFLUOUS,INTERESTING
0,ABANDON,ABLE,ABEYANCE,ABOVEMENTIONED,ABIDE,AEGIS,AGGRESSIVE
1,ABANDONED,ABUNDANCE,ABEYANCES,ABROGATE,ABIDING,AMORPHOUS,AGGRESSIVELY
2,ABANDONING,ABUNDANT,ALMOST,ABROGATED,BOUND,ANTICIPATORY,AGGRESSIVENESS
3,ABANDONMENT,ACCLAIMED,ALTERATION,ABROGATES,BOUNDED,APPERTAINING,ASBESTOS
4,ABANDONMENTS,ACCOMPLISH,ALTERATIONS,ABROGATING,COMMIT,ASSIMILATE,AUGUST


In [4]:
dictionary = dictionary.fillna('')

In [5]:
wordCountSentiment.lemmatizeDictionary(dictionary)
dictionary.head()

,NEGATIVE,POSITIVE,UNCERTAINTY,LITIGIOUS,CONSTRAINING,SUPERFLUOUS,INTERESTING
0,abandon,abl,abey,abovement,abid,aegi,aggress
1,abandon,abund,abey,abrog,abid,amorph,aggress
2,abandon,abund,almost,abrog,bound,anticipatori,aggress
3,abandon,acclaim,alter,abrog,bound,appertain,asbesto
4,abandon,accomplish,alter,abrog,commit,assimil,august


In [6]:
wordCountSentiment.getSentiment('Apple is designing its own power-management chips for use in iPhones as early as 2018, the Nikkei business daily reported on Thursday, triggering a more than 20 percent slide in shares of supplier Dialog Semiconductor.  If confirmed, the move would reduce Apple is dependence on the Anglo-German chipmaker, which itself is heavily reliant on the smartphone industry and has been trying to diversify its customer base.  Investors are particularly jittery after Apple said in April that it planned to replace graphics chip supplier Imagination Technologies, sending the London-listed stock down 70 percent in a single session.',dictionary)

{'NEGATIVE': 0,
 'POSITIVE': 1,
 'UNCERTAINTY': 1,
 'LITIGIOUS': 1,
 'CONSTRAINING': 1,
 'SUPERFLUOUS': 0,
 'INTERESTING': 0}

In [7]:
# 
tickers=['AAPL','AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX','ADBE','AMGN','TXN','AVGO','PYPL','GILD','COST','QCOM']       


In [ ]:
## -------- EXTRACT SENTIMENT SINGLE NEWS 

mypath = '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'
tickFiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for ticker in tickFiles:
    print(ticker)
    news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'+ticker)
    news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)
    news = news.drop(['Unnamed: 0'], axis=1)
    notNaN = news['SUMMARY'].dropna(how='any').index.tolist()
    NaN = []
    for i in news.index:
        if(i not in notNaN):
            NaN.append(i)
    news = news.drop(NaN)
    # #news.head(10)
    news = news.reset_index(drop=True)
    news['PUBLICATION_DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['PUBLICATION_DATE']]
    


    sentiment = pd.DataFrame(columns=['PUBLICATION_DATE','CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING'])

    for i, row in news.iterrows():
        try:
            sent = wordCountSentiment.getSentiment(news['SUMMARY'][i], dictionary)
            sentiment.loc[i] = {'PUBLICATION_DATE':news['PUBLICATION_DATE'][i], 'CONSTRAINING':sent['CONSTRAINING'],
                            'LITIGIOUS': sent['LITIGIOUS'],'NEGATIVE': sent['NEGATIVE'], 'POSITIVE': sent['POSITIVE'],
                            'UNCERTAINTY': sent['UNCERTAINTY'],'SUPERFLUOUS': sent['SUPERFLUOUS'],'INTERESTING': sent['INTERESTING']}
        except:
            print("ERROR news "+str(i))

  
    sentiment.to_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/Sentiment/'+ticker)

In [ ]:
# EXTRACT SENTIMENT AGGREGATED BY TIMESPAN

mypath = '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'
tickFiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for ticker in tickers: 
    print(ticker)
    news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'+ticker+'.csv')
    count =1
    while str(ticker)+str(count)+'.csv' in tickFiles:
        print(ticker+str(count))
        newsTemp = pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/' + ticker +str(count)+'.csv')
        news = pd.concat([news, newsTemp])
        count+=1
    news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)
    news = news.drop(['Unnamed: 0'], axis=1)
    notNaN = news['SUMMARY'].dropna(how='any').index.tolist()
    NaN = []
    for i in news.index:
        if(i not in notNaN):
            NaN.append(i)
    news = news.drop(NaN)
    #news.head(10)
    news = news.reset_index(drop=True)
    news['PUBLICATION_DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['PUBLICATION_DATE']]
    
    
    timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+ticker+".csv")
    #timeSpan.rename(columns={'Unnamed: 0':'date'}, inplace= True)
    timeSpan = timeSpan['date'].tolist()
    # This dataset is already GMT+0
    timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]
    #timeSpan = [row-timedelta(hours=8) for row in timeSpan]


    sentiment = pd.DataFrame(columns=['initTime','CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING', 'NUM_NEWS'])


    #Prendiamo intervallo massimo dove ci sono entrambi i dataset:
    #massimo della data iniziale tra i due
    #minimo della data finale tra i due
    initDate = max(timeSpan[0], news['PUBLICATION_DATE'][0])
    finalDate = min(timeSpan[len(timeSpan)-1], news['PUBLICATION_DATE'][len(news)-1])
    last_news={'initTime':0,'CONSTRAINING': 0, 'LITIGIOUS': 0, 'NEGATIVE': 0, 'POSITIVE': 0, 'UNCERTAINTY': 0}



    i = 0;
    j = 0

    # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    init = j
    while(news['PUBLICATION_DATE'][i] <= initDate):
        i+=1

    print("Start: "+str(initDate) +" i: "+str(i)+" j: "+str(j))
    print("End: "+str(finalDate))

    #ALLINEAMENTO FINE DENTRO I WHILE
    while( news['PUBLICATION_DATE'][i] < finalDate and timeSpan[j] < finalDate ):
        num_sentiment = 0
        initTime = timeSpan[j]
        normal_sum = {'CONSTRAINING': 0, 'LITIGIOUS': 0, 'NEGATIVE': 0, 'POSITIVE': 0, 'UNCERTAINTY': 0,'SUPERFLUOUS':0,'INTERESTING':0}
        while(i<len(news)-1 and timeSpan[j] > news['PUBLICATION_DATE'][i]):
            if not (timeSpan[j] > news['PUBLICATION_DATE'][i] and timeSpan[j-1] <= news['PUBLICATION_DATE'][i]):
                print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['PUBLICATION_DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                sent = wordCountSentiment.getSentiment(news['SUMMARY'][i], dictionary)
                normal_sum['CONSTRAINING'] += sent['CONSTRAINING']
                normal_sum['LITIGIOUS'] += sent['LITIGIOUS']
                normal_sum['NEGATIVE'] += sent['NEGATIVE']
                normal_sum['POSITIVE'] += sent['POSITIVE']
                normal_sum['UNCERTAINTY'] += sent['UNCERTAINTY']
                normal_sum['SUPERFLUOUS'] += sent['SUPERFLUOUS']
                normal_sum['INTERESTING'] += sent['INTERESTING']
                num_sentiment +=1
            except:
                print("ERROR news "+str(i))
            i+=1
        for key, value in normal_sum.items():
            if(num_sentiment != 0):
                normal_sum[key] /=num_sentiment
        j+=1
        sentiment.loc[j] = {'initTime':initTime, 'CONSTRAINING':normal_sum['CONSTRAINING'],
                            'LITIGIOUS': normal_sum['LITIGIOUS'],
                            'NEGATIVE': normal_sum['NEGATIVE'], 'POSITIVE': normal_sum['POSITIVE'],
                            'UNCERTAINTY': normal_sum['UNCERTAINTY'], 'SUPERFLUOUS': normal_sum['SUPERFLUOUS'],
                            'INTERESTING': normal_sum['INTERESTING'], 'NUM_NEWS':num_sentiment }
    
    
    
    end = j
    if(len(sentiment) != end - init):
        print(len(sentiment))
        print(end - init + 1)
        assert False

    
    sentiment.to_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentFullAggregatedHourly/'+ticker+'.csv')

In [ ]:
comp.tail(100)

In [8]:
# EXTRACT SENTIMENT OF TITLE AGGREGATED BY TIMESPAN

mypath = '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'
tickFiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for ticker in tickers: 
    print(ticker)
    news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'+ticker+'.csv')
    count =1
    while str(ticker)+str(count)+'.csv' in tickFiles:
        print(ticker+str(count))
        newsTemp = pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/' + ticker +str(count)+'.csv')
        news = pd.concat([news, newsTemp])
        count+=1
    news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)
    news = news.drop(['Unnamed: 0'], axis=1)
    notNaN = news['TITLE'].dropna(how='any').index.tolist()
    NaN = []
    for i in news.index:
        if(i not in notNaN):
            NaN.append(i)
    news = news.drop(NaN)
    #news.head(10)
    news = news.reset_index(drop=True)
    news['PUBLICATION_DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['PUBLICATION_DATE']]
    
    
    timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+ticker+".csv")
    #timeSpan.rename(columns={'Unnamed: 0':'date'}, inplace= True)
    timeSpan = timeSpan['date'].tolist()
    # This dataset is already GMT+0
    timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]
    #timeSpan = [row-timedelta(hours=8) for row in timeSpan]


    sentiment = pd.DataFrame(columns=['initTime','CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING', 'NUM_NEWS'])


    #Prendiamo intervallo massimo dove ci sono entrambi i dataset:
    #massimo della data iniziale tra i due
    #minimo della data finale tra i due
    initDate = max(timeSpan[0], news['PUBLICATION_DATE'][0])
    finalDate = min(timeSpan[len(timeSpan)-1], news['PUBLICATION_DATE'][len(news)-1])
    last_news={'initTime':0,'CONSTRAINING': 0, 'LITIGIOUS': 0, 'NEGATIVE': 0, 'POSITIVE': 0, 'UNCERTAINTY': 0}



    i = 0;
    j = 0

    # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    init = j
    while(news['PUBLICATION_DATE'][i] <= initDate):
        i+=1

    print("Start: "+str(initDate) +" i: "+str(i)+" j: "+str(j))
    print("End: "+str(finalDate))

    #ALLINEAMENTO FINE DENTRO I WHILE
    while( news['PUBLICATION_DATE'][i] < finalDate and timeSpan[j] < finalDate ):
        num_sentiment = 0
        initTime = timeSpan[j]
        normal_sum = {'CONSTRAINING': 0, 'LITIGIOUS': 0, 'NEGATIVE': 0, 'POSITIVE': 0, 'UNCERTAINTY': 0,'SUPERFLUOUS':0,'INTERESTING':0}
        while(i<len(news)-1 and timeSpan[j] > news['PUBLICATION_DATE'][i]):
            if not (timeSpan[j] > news['PUBLICATION_DATE'][i] and timeSpan[j-1] <= news['PUBLICATION_DATE'][i]):
                print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['PUBLICATION_DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                sent = wordCountSentiment.getSentiment(news['TITLE'][i], dictionary)
                normal_sum['CONSTRAINING'] += sent['CONSTRAINING']
                normal_sum['LITIGIOUS'] += sent['LITIGIOUS']
                normal_sum['NEGATIVE'] += sent['NEGATIVE']
                normal_sum['POSITIVE'] += sent['POSITIVE']
                normal_sum['UNCERTAINTY'] += sent['UNCERTAINTY']
                normal_sum['SUPERFLUOUS'] += sent['SUPERFLUOUS']
                normal_sum['INTERESTING'] += sent['INTERESTING']
                num_sentiment +=1
            except:
                print("ERROR news "+str(i))
            i+=1
        for key, value in normal_sum.items():
            if(num_sentiment != 0):
                normal_sum[key] /=num_sentiment
        j+=1
        sentiment.loc[j] = {'initTime':initTime, 'CONSTRAINING':normal_sum['CONSTRAINING'],
                            'LITIGIOUS': normal_sum['LITIGIOUS'],
                            'NEGATIVE': normal_sum['NEGATIVE'], 'POSITIVE': normal_sum['POSITIVE'],
                            'UNCERTAINTY': normal_sum['UNCERTAINTY'], 'SUPERFLUOUS': normal_sum['SUPERFLUOUS'],
                            'INTERESTING': normal_sum['INTERESTING'], 'NUM_NEWS':num_sentiment }
    
    
    
    end = j
    if(len(sentiment) != end - init):
        print(len(sentiment))
        print(end - init + 1)
        assert False

    
    sentiment.to_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentTitleAggregatedHourly/'+ticker+'.csv')

AAPL
AAPL1
AAPL2
Start: 2017-04-03 17:27:34 i: 1 j: 3817
End: 2018-06-21 23:20:00
AMZN
AMZN1
AMZN2
Start: 2017-04-26 13:00:00 i: 1 j: 3925
End: 2018-06-22 02:35:18
GOOGL
GOOGL1
Start: 2017-05-21 15:00:05 i: 1 j: 4051
End: 2018-06-22 00:14:21
MSFT
MSFT1
Start: 2016-04-22 01:05:08 i: 1 j: 2150
End: 2018-06-22 00:10:12
FB
FB1
FB2
Start: 2016-04-28 00:17:37 i: 1 j: 2178
End: 2018-06-21 23:30:00
INTC
Start: 2017-02-15 14:20:32 i: 1 j: 3589
End: 2018-06-22 02:45:56
CSCO
Start: 2015-02-11 22:25:31 i: 1 j: 56
End: 2018-06-20 19:12:55
CMCSA
Start: 2015-04-21 17:28:40 i: 1 j: 382
End: 2018-06-21 22:32:37
NVDA
Start: 2017-01-19 22:49:53 i: 1 j: 3463
End: 2018-06-21 22:16:10
NFLX
NFLX1
Start: 2016-01-05 13:01:49 i: 1 j: 1625
End: 2018-06-21 22:54:33
ADBE


FileNotFoundError: File b'/home/simone/Desktop/NLFF/indexes/indexesADBE.csv' does not exist